In [1]:
import struct
import time
from pathlib import Path

from matplotlib import pyplot as plt
from torchvision import transforms
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
def load_label(filename):
    with open(filename, "rb") as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        return np.fromfile(flbl, dtype=np.int8)


def load_data(filename):
    with open(filename, "rb") as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        return np.fromfile(fimg, dtype=np.uint8).reshape(num, rows, cols)


BASE = Path("../mnist")

train_images = torch.tensor(load_data(BASE / "train-images-idx3-ubyte")).long()
train_labels = torch.tensor(load_label(BASE / "train-labels-idx1-ubyte")).long()
test_images = torch.tensor(load_data(BASE / "t10k-images-idx3-ubyte")).long()
test_labels = torch.tensor(load_label(BASE / "t10k-labels-idx1-ubyte")).long()

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])
torch.Size([10000, 28, 28])
torch.Size([10000])


In [3]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size=10):
        super().__init__()
        self.layer1 = nn.Linear(input_size, 16, bias=False)
        self.layer2 = nn.Linear(16, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        return F.log_softmax(x, dim=1)

In [4]:
def normalize_input(images, size, depth):
    R = transforms.Resize((size, size))
    images = R(images).long()
    images = torch.flatten(images, start_dim=1)
    return images >> (8 - depth)

In [5]:
BATCH_SIZE = 200
INPUT_SIZE = 8
INPUT_DEPTH = 4
INPUT_COUNT = train_labels.shape[0]

train_inputs = normalize_input(train_images, INPUT_SIZE, INPUT_DEPTH)
test_inputs = normalize_input(test_images, INPUT_SIZE, INPUT_DEPTH)
train_output = F.one_hot(train_labels, num_classes=10)

net = MLP(INPUT_SIZE**2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


start = time.time()
for epoch in range(128):
    running_loss = 0
    num_batches = 0
    shuffled_indices = torch.randperm(INPUT_COUNT)
    for count in range(0, INPUT_COUNT, BATCH_SIZE):
        indices = shuffled_indices[count : count + BATCH_SIZE]

        batch_inputs = train_inputs[indices]
        batch_output = train_output[indices]

        # forward and backward pass
        optimizer.zero_grad()
        loss = criterion(net(batch_inputs.float()), batch_output.float())
        loss.backward()
        optimizer.step()

        # compute some stats
        running_loss += loss.detach().item()
        num_batches += 1

        total_loss = running_loss / num_batches
        elapsed_time = time.time() - start

        if num_batches % 100 == 0:
            # scheduler.step()
            with torch.no_grad():
                outputs = net(test_inputs.float())
                correct = sum(outputs.argmax(dim=1) == test_labels).item()

            print(
                {
                    "epoch": epoch,
                    "batches": num_batches,
                    "elapsed_time": round(elapsed_time, 2),
                    "total_loss": total_loss,
                    "accuracy": round(correct / len(test_inputs) * 100, 2),
                }
            )

{'epoch': 0, 'batches': 100, 'elapsed_time': 0.51, 'total_loss': 2.092724039554596, 'accuracy': 41.42}
{'epoch': 0, 'batches': 200, 'elapsed_time': 1.12, 'total_loss': 1.7636020278930664, 'accuracy': 66.19}
{'epoch': 0, 'batches': 300, 'elapsed_time': 1.72, 'total_loss': 1.5058273615439732, 'accuracy': 73.68}
{'epoch': 1, 'batches': 100, 'elapsed_time': 2.32, 'total_loss': 0.8224061280488968, 'accuracy': 76.86}
{'epoch': 1, 'batches': 200, 'elapsed_time': 2.73, 'total_loss': 0.78771329164505, 'accuracy': 78.42}
{'epoch': 1, 'batches': 300, 'elapsed_time': 3.17, 'total_loss': 0.7570631332198778, 'accuracy': 79.84}
{'epoch': 2, 'batches': 100, 'elapsed_time': 3.68, 'total_loss': 0.665952125787735, 'accuracy': 80.55}
{'epoch': 2, 'batches': 200, 'elapsed_time': 4.33, 'total_loss': 0.6505167734622955, 'accuracy': 81.54}
{'epoch': 2, 'batches': 300, 'elapsed_time': 5.09, 'total_loss': 0.6389342275261879, 'accuracy': 81.8}
{'epoch': 3, 'batches': 100, 'elapsed_time': 5.87, 'total_loss': 0.60

In [6]:
def predict(inputs, w1, w2, b):
    hid = (inputs @ w1).astype(np.int16)
    hid = hid * (hid > 0)
    out = hid @ w2 + b
    return np.argmax(out, axis=1)


def clip(arr, scale, bits):
    imax = 2 ** (bits - 1) - 1
    imin = -(2 ** (bits - 1))

    return np.clip(arr * scale, imin, imax)

In [7]:
data = []

weight1 = net.layer1.weight.detach().numpy().transpose()
weight2 = net.layer2.weight.detach().numpy().transpose()
bias = net.layer2.bias.detach().numpy()

for i in range(0, 32):
    SCALE = 1 << i

    img = test_inputs.numpy().astype(np.unt8)
    w1 = clip(weight1, SCALE, 8).astype(np.int8)
    w2 = clip(weight2, SCALE, 8).astype(np.int8)
    b = clip(bias, SCALE, 16).astype(np.int16)

    pred = predict(img, w1, w2, b)
    data.append(
        {
            "pow": i,
            "scale": SCALE,
            "accuracy": sum(pred == test_labels.numpy()) / len(test_labels),
        }
    )

stat = pd.DataFrame(data)
scale = stat.set_index("scale")["accuracy"].idxmax()
stat[stat["scale"] == scale]

,pow,scale,accuracy
5,5,32,0.8797


In [8]:
fig, ax = plt.subplots(figsize=(8, 5), dpi=120)
ax.scatter(stat["scale"], stat["accuracy"] * 100)
ax.axhline(y=85, color="orange", linestyle="dashed")
ax.set_xscale("log", base=2)
ax.set_xlabel("Scale")
ax.set_xlim(1, 2**32)
ax.set_ylabel("Accuracy (%)")
ax.set_ylim(0, 100)
fig.tight_layout()
fig.savefig("customized.pdf")
fig.savefig("customized.png")

In [9]:
w1 = clip(weight1, scale, 8).astype(np.int8)
w2 = clip(weight2, scale, 8).astype(np.int8)
b = clip(bias, scale, 8).astype(np.int8)

# inputs = inputs.astype(np.uint8)
# w1 = w1.astype(np.int8)
# w2 = w2.astype(np.int8)
# b = b.astype(np.int16)

np.save("test_data_custom", test_inputs)
np.save("test_label_custom", test_labels)
np.save("model_weight1_custom", w1)
np.save("model_weight2_custom", w2)
np.save("model_bias_custom", b)